In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from dateutil.relativedelta import relativedelta
from skforecast.ForecasterAutoreg import ForecasterAutoreg
plt.style.use('seaborn-v0_8-darkgrid')

from predict import Predict

In [2]:
df = pd.read_csv('database/DIM_PLAYER_GAMES.csv')
test = list(df['PLAYER'].unique()[:100])
data = df[df['PLAYER'].isin(test)]


In [3]:
w = Predict(df=df, player = 'Sam LaPorta', steps=17, regressor=RandomForestRegressor, lags=17)
w.predict()


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(


ValueError: Found input variables with inconsistent numbers of samples: [10, 17]

In [ ]:
results = [
    Predict(
        df=df, player=name, steps=17, regressor=RandomForestRegressor, lags=17
    ).predict()
    for name in test
]

c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(


okay


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(


okay


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

...

c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(


okay


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

okay
okay


ValueError: The maximum lag (17) must be less than the length of the series (8).

In [ ]:
np.mean(df_res['SEASON MAPE']), np.mean(df_res['GAME RSME'])

(nan, nan)

In [ ]:
z = Predict(df=data, players = test, steps=17, regressor=GradientBoostingRegressor, lags=17)
df_res2 = z.predict_all()
np.mean(df_res2['SEASON MAPE']), np.mean(df_res2['GAME RSME'])


c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.date_range(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data["FORECAST_DATE"] = pd.to_datetime(
c:\Users\012356.CORPORATE\OneDrive - TRIMEDX Holdings, LLC\Desktop\FF_DB\predict.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

(nan, nan)